In [141]:
from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
import numpy as np, pandas as pd
# import seaborn as sns

In [142]:
# how to extract the film's genre
movie_containers[0].p.find('span', class_= "genre").text.replace("\n", "").strip()

'Action, Drama, Mystery'

In [143]:
# Note this takes about 40 min to run if np.arange is set to 9951 as the stopping point.

pages = np.arange(1,20, 1) # Last time I tried, I could only go to 10000 items because after that the URI has no discernable pattern to combat webcrawlers; I just did 4 pages for demonstration purposes. You can increase this for your own projects.
pages = [(pages[i] + 50) for i in range (len(pages))]
headers = {'Accept-Language': 'en-US,en;q=0.8'} # If this is not specified, the default language is Mandarin

#initialize empty lists to store the variables scraped
titles = []
years = []
ratings = []
genres = []
runtimes = []
imdb_ratings = []
imdb_ratings_standardized = []
metascores = []
votes = []
directors = []
response = get("https://www.imdb.com/search/title?genres=sci-fi&"
                  + "start="
                  + str(pages[0])
                  + "&explore=title_type,genres&ref_=adv_prv", headers=headers)
response
page_html = BeautifulSoup(response.text, 'html.parser')
page_html
movie_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
# movie_containers
# movie_containers[1].find_all('div', class_ = "Stars")

In [144]:
# how to extarct the movie name
movie_containers[0].h3.find("a").text

'Blade Runner 2049'

In [145]:
# the imdb rating
movie_containers[0].strong.text

'8.0'

In [146]:
# the director name
# here we use the find method afforded by the bs4 method not find of the string class
movie_containers[0].find("p", class_='').find("a").text

'Denis Villeneuve'

In [147]:
star_elemnts = movie_containers[0].find("p", class_ = "").find_all("a")
names = [s.text for s in star_elemnts]
star_names = names[1:]
star_names
pages

[51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]

In [148]:
for page in pages:
  
   #get request for sci-fi
   response = get("https://www.imdb.com/search/title?genres=sci-fi&"
                  + "start="
                  + str(page)
                  + "&explore=title_type,genres&ref_=adv_prv", headers=headers)
  
   sleep(randint(8,15))
   
   #throw warning for status codes that are not 200
   if response.status_code != 200:
       warn('Request: {}; Status code: {}'.format(requests, response.status_code))

   #parse the content of current iteration of request
   page_html = BeautifulSoup(response.text, 'html.parser')
      
   movie_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
  
   #extract the 50 movies for that page
   for container in movie_containers:

       #conditional for all with metascore, this doesn t consider movies without metascore
       if container.find('div', class_ = 'ratings-metascore') is not None:

           #title
           title = container.h3.find("a").text
           titles.append(title)

           if container.h3.find('span', class_= 'lister-item-year text-muted unbold') is not None:
            
             #year released
             year = container.h3.find('span', class_= 'lister-item-year text-muted unbold').text # remove the parentheses around the year and make it an integer
             years.append(year)

           else:
             years.append(None) # each of the additional if clauses are to handle type None data, replacing it with an empty string so the arrays are of the same length at the end of the scraping

           # if container.p.find('span', class_ = 'certificate') is not None:
            
           #   #rating
           #   rating = container.p.find('span', class_= 'certificate').text
           #   ratings.append(rating)

           # else:
           #   ratings.append("")

           if container.p.find('span', class_ = 'genre') is not None:
            
             #genre
             genre = container.p.find('span', class_ = 'genre').text.replace("\n", "").rstrip().split(',') # remove the whitespace character, strip, and split to create an array of genres
             genres.append(genre)
          
           else:
             genres.append("")

           if container.p.find('span', class_ = 'runtime') is not None:

             #runtime
             time = int(container.p.find('span', class_ = 'runtime').text.replace(" min", "")) # remove the minute word from the runtime and make it an integer
             runtimes.append(time)

           else:
             runtimes.append(None)

           if float(container.strong.text) is not None:

             #IMDB ratings
             imdb = float(container.strong.text) # non-standardized variable
             imdb_ratings.append(imdb)

           else:
             imdb_ratings.append(None)

           if container.find('span', class_ = 'metascore').text is not None:

             #Metascore
             m_score = int(container.find('span', class_ = 'metascore').text) # make it an integer
             metascores.append(m_score)

           else:
             metascores.append(None)
               # 
           if container.find("p", class_='').find("a").text is not None:

             #Director
             d = container.find("p", class_='').find("a").text
             directors.append(d)
           else:
             directors.append("Not found")
           if container.find('span', attrs = {'name':'nv'})['data-value'] is not None:

             #Number of votes
             vote = int(container.find('span', attrs = {'name':'nv'})['data-value'])
             votes.append(vote)

           else:
               votes.append(None)
# titles
# directors

In [150]:
# titles_data = pd.Series(titles, name = "titles_data")
# directors_data = pd.Series(directors, name = "directors_data")
titles_data = pd.DataFrame(titles, columns= ["Title"])
directors_data = pd.DataFrame(directors, columns=["Director"])
runtimes_data = pd.DataFrame(runtimes, columns=["Runtime"])
imdb_ratings_data = pd.DataFrame(imdb_ratings, columns=["Imdb Rating"])
# meta_data = pd.merge(titles_data, directors_data, how='left' )
meta_data = pd.concat([titles_data, directors_data, runtimes_data, imdb_ratings_data], axis = 1)
meta_data

,Title,Director,Runtime,Imdb Rating
0,Blade Runner 2049,Denis Villeneuve,164,8.0
1,65,Scott Beck,93,5.4
2,Guardians of the Galaxy Vol. 2,James Gunn,136,7.6
3,Spider-Man: No Way Home,Jon Watts,148,8.2
4,Jurassic Park,Steven Spielberg,127,8.2
...,...,...,...,...
450,Don't Look Up,Adam McKay,138,7.2
451,Blade Runner,Ridley Scott,117,8.1
452,The Terminator,James Cameron,107,8.1
453,2001: A Space Odyssey,Stanley Kubrick,149,8.3
